In [1]:
import random
import statistics
from typing import List
from scipy import stats

In [6]:
# Constants
TRACK_LENGTH = 100
OBSTACLE_COUNT = 5
BOOST_COUNT = 3
SUCCESS_TIME_THRESHOLD = 15

# Movement ranges per strategy
MOVE_RANGES = {
    'aggressive': (8, 15),
    'cautious': (1, 7),
    'neutral': (3, 10),
}

# Penalty for colliding with an obstacle
OBSTACLE_PENALTY = {
    'aggressive': 15,
    'cautious': 1,
    'neutral': 6,
}

# Speed boost amount
BOOST_AMOUNT = {
    'aggressive': 15,
    'cautious': 5,
    'neutral': 10
}

def generate_positions(count: int,
                       start: int = 10, 
                       end: int = 90) -> List[int]:
    """
    Returen 'count' random integer positions in [start, end].
    """
    return [random.randint(start, end) for _ in range(count)]

def predict_strategy(outcome: str) -> str:
    """
    Given an outcome ('success' or 'fail'), pick a retro-causal strategy.
    """
    if outcome == 'success':
        print("Anakin senses a clear path for speed boosting through the Force! -> aggressive")
        return 'aggressive'
    
    print("Anakin senses a disturbance in the Force... -> cautious:")
    return 'cautious'

def run_race(strategy: str,
             obstacles: List[int],
             boosts: List[int],
             track_length: int = TRACK_LENGTH,
             debug: bool = False) -> int:
    """
    Simulate a single podrace using 'strategy'. Returns the number of time steps taken to finish the race.
    If debug=True, prints step-by-step information.
    """
    position = 0
    time = 0
    
    move_min, move_max = MOVE_RANGES[strategy]
    obstacle_penalty = OBSTACLE_PENALTY[strategy]
    boost_amount = BOOST_AMOUNT[strategy]
    
    if debug:
        print(f"\n== Starting race [{strategy}] ==")
        print(f"Track length: {track_length}")
        print(f"Obstacle positions: {obstacles}")
        print(f"Boost positions: {boosts}\n")
        
    while position < track_length:
        # 1) move pod
        step = random.randint(move_min, move_max)
        position += step
        time += 1
            
        # 2) obstacle penalty?
        if position in obstacles:
            position -= obstacle_penalty
            if debug:
                print(f"Hit obstacle @ {position + penalty} -> -{obstacle_penalty}, nwo {position}")
                
        # 3) speed boost?
        if position in boosts:
            position += boost_amount
            if debug:
                print(f"Speed boost @ {position - boost_amount} -> +{boost_amount}")
                
        # 4) debug print
        if debug:
            print(f"Time {time:2d}: pos {position}")
    
    return time

def demo_race() -> None:
    """
    Run one verbose demo: neutral start -> force prediction -> second run.
    """
    
    obstacles = generate_positions(OBSTACLE_COUNT)
    boosts = generate_positions(BOOST_COUNT)
    
    # Neutral run
    t0 = run_race('neutral', obstacles, boosts, debug=True)
    outcome0 = 'success' if t0 < SUCCESS_TIME_THRESHOLD else 'fail'
    print(f"\nResult of neutral run: time={t0}, outcome={outcome0}")
    
    # Retro-causal strategy
    strategy = predict_strategy(outcome0)
    t1 = run_race(strategy, obstacles, boosts, debug=True)
    print(f"\nResult of {strategy} run: time={t1}\n")
    
def simulate_races(rounds: int = 1000, debug: bool = False) -> None:
    """
    Run 'rounds' races without strategy vs. with retro-causal strategy, then do a t-test on the finishing times.
    """
    
    neutral_times = []
    retrocausal_times = []
    
    for _ in range(rounds):
        obstacles = generate_positions(OBSTACLE_COUNT)
        boosts = generate_positions(BOOST_COUNT)
        
        # 1) baseline (neutral) run
        t_neutral = run_race('neutral', obstacles, boosts, debug=False)
        neutral_times.append(t_neutral)
        
        # 2) predict and run with retro-causal strategy
        outcome = 'success' if t_neutral < SUCCESS_TIME_THRESHOLD else 'fail'
        strat = predict_strategy(outcome)
        t_retrocausal = run_race(strat, obstacles, boosts, debug=False)
        retrocausal_times.append(t_retrocausal)
        
    # Compute summary statistics
    mean_neutral = statistics.mean(neutral_times)
    mean_retrocausal = statistics.mean(retrocausal_times)
    std_neutral = statistics.stdev(neutral_times)
    std_retrocausal = statistics.stdev(retrocausal_times)
    
    # Independent t-test
    t_stat, p_val = stats.ttest_ind(neutral_times, retrocausal_times)
    
    print(f"\n=== After {rounds} simulated races ===")
    print(f"Neutral strategy: mean={mean_neutral:.2f}, std={std_neutral:.2f}")
    print(f"Retro-causal strategy: mean={mean_retrocausal:.2f}, std={std_retrocausal:.2f}")
    print(f"T-test: t={t_stat:.3f}, p={p_val:.3f}")
    if p_val < 0.05:
        print("The difference is statistically significant!")
    else:
        print("No significant difference detected.")

if __name__ == "__main__":
    # 1) One demo run with detailed logs
    demo_race()
    
    #2) Bulk simulaiton (quiet) + statistical test
    simulate_races(rounds=1000, debug=False)                    


== Starting race [neutral] ==
Track length: 100
Obstacle positions: [29, 68, 11, 27, 13]
Boost positions: [80, 30, 45]

Time  1: pos 6
Time  2: pos 16
Time  3: pos 25
Time  4: pos 32
Time  5: pos 42
Time  6: pos 51
Time  7: pos 60
Time  8: pos 65
Time  9: pos 75
Time 10: pos 84
Time 11: pos 87
Time 12: pos 90
Time 13: pos 96
Time 14: pos 101

Result of neutral run: time=14, outcome=success
Anakin senses a clear path for speed boosting through the Force! -> aggressive

== Starting race [aggressive] ==
Track length: 100
Obstacle positions: [29, 68, 11, 27, 13]
Boost positions: [80, 30, 45]

Time  1: pos 8
Time  2: pos 22
Time  3: pos 34
Time  4: pos 42
Time  5: pos 57
Time  6: pos 71
Time  7: pos 84
Time  8: pos 99
Time  9: pos 109

Result of aggressive run: time=9

Anakin senses a disturbance in the Force... -> cautious:
Anakin senses a clear path for speed boosting through the Force! -> aggressive
Anakin senses a disturbance in the Force... -> cautious:
Anakin senses a clear path for 